In [ ]:

import json

with open("data.json") as json_data:
    data = json.load(json_data)
print(data)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
words = []
pattern_words_with_tag = []
classes = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        pattern_words = nltk.word_tokenize(pattern)

        words.extend(pattern_words)
        pattern_words_with_tag.append((pattern_words, intent["tag"])) # Tuple type

        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [ ]:
print(words)

In [ ]:
#a list of tuples
print(pattern_words_with_tag)

In [ ]:
print(classes)

In [ ]:
#  Clean chat data for machine learning
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

words_lowercase = [stemmer.stem(word.lower()) for word in words]

# converting to set will remove duplicate elements
unique_words = sorted(list(set(words_lowercase)))
print(unique_words)

In [ ]:
empty_output = [0] * len(classes)
print(empty_output)

output_row = list(empty_output)
print(output_row)


In [ ]:
#  Build bag of words for ML model

# print(documents)
empty_output = [0] * len(classes)
# print(empty_output)

training_data = []

for tuple in pattern_words_with_tag:
    bag_of_words = []

    # Tuple: ([pattern_words], tag)
    pattern_words = tuple[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for unique_word in unique_words:
        bag_of_words.append(1) if unique_word in pattern_words else bag_of_words.append(0)

    output_row = list(empty_output)
    output_row[classes.index(tuple[1])] = 1
    training_data.append([bag_of_words, output_row])

#print(pattern_words)

#training_data: a list of a tuple of ([bag of words], [bag of tag])
#https://en.wikipedia.org/wiki/Bag-of-words_model
print(training_data)



In [ ]:
print(len(training_data))

In [ ]:
#  Split data for machine learning

import random
random.shuffle(training_data)

#print(training_data)
#print(type(training_data))

import numpy
training_numpy = numpy.array(training_data, dtype=object)

#print(training_numpy)
#print(type(training_numpy))

train_X = list(training_numpy[:,0]) # to access column 0

print(train_X)
#print(len(train_X))

train_Y = list(training_numpy[:,1]) # to access column 1
print(train_Y)

In [ ]:
#  Build a TensorFlow machine learning model for chat

import tflearn 
neural_network = tflearn.input_data(shape = [None, len(train_X[0])])
print(neural_network)

neural_network = tflearn.fully_connected(neural_network, 8)
print(neural_network)

neural_network = tflearn.fully_connected(neural_network, 8)
print(neural_network)

neural_network = tflearn.fully_connected(neural_network, len(train_Y[0]), activation="softmax")
print(neural_network)

neural_network = tflearn.regression(neural_network)
print(neural_network)

model = tflearn.DNN(neural_network)
print(model)

model.fit(train_X, train_Y, n_epoch = 2000, batch_size = 8, show_metric = True)

In [ ]:
# Test chatbot machine learning model
model.save("chatbot_dnn.tflearn")


In [ ]:
model.load("chatbot_dnn.tflearn")

print(model)

question = "Do you sell any coding course?"

def process_question(question):
    question_tokenized = nltk.word_tokenize(question)
    question_stemmed = [stemmer.stem(word.lower()) for word in question_tokenized]

    bag = [0] * len(unique_words)

    for stem in question_stemmed:
        for index, word in enumerate(unique_words):
            if word == stem:
                bag[index] = 1

    return(numpy.array(bag))

processed_question = process_question(question)
print(len(processed_question))

In [ ]:
prediction = model.predict([processed_question])
print(prediction)
#print(classes)

In [ ]:
def categorize(prediction):
    prediction_top = [[index, result] for index,result in enumerate(prediction) if result > 0.5]    
    prediction_top.sort(key=lambda x: x[1], reverse = True)     
    result = []

    for prediction_value in prediction_top:
        result.append((classes[prediction_value[0]], prediction_value[1]))    
    return result

prediction_result = categorize(prediction[0])
print(prediction_result)


    

In [ ]:
def chatbot(question):
    prediction = model.predict([process_question(question)])   
    result = categorize(prediction[0])    
    return result




In [ ]:
# Pick a chatbot response in top category

user_input = input("Do you have a question for me?\n")
print(user_input)
def respond_to_input(user_input):   
    question_category = chatbot(user_input)    
    if question_category:
        for intent in data["intents"]:
            if intent["tag"] == question_category[0][0]:
                return random.choice(intent["responses"])
    else:
        print("Could you please clarify your question ?")
        
respond_to_input(user_input)



In [ ]:
for i in range(4):
    user_input = input("Do you have a question for me?\n")
    response = respond_to_input(user_input)
    print(response)